# Option 2. Finding distance by focal length

In [1]:
import utils
import torch
import math
import numpy as np 
from PIL import Image
import cv2
import os
from pathlib import *
import shutil
import pandas as PD
import pillow_heif
import matplotlib.pyplot as plt
from exif import Image as exif
%matplotlib inline
PD.options.display.expand_frame_repr = False

In [2]:
display = utils.notebook_init() 
print('Cuda available?', torch.cuda.is_available())
print('At which number CUDA:', torch.cuda.current_device())
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # without this there will be an error

YOLOv5  2023-3-13 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)


Setup complete  (12 CPUs, 32.0 GB RAM, 764.8/919.6 GB disk)
Cuda available? True
At which number CUDA: 0


## Load an appropriate model

In [3]:
# Load model
#model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs/train/bw/exp4/weights/last.pt') # works, but loads the BEST every time
#model = torch.hub.load('/home/jovyan/car-distance-ptod/yolov5', 'custom',  path='runs/train/exp/weights/best.pt') # works, but loads every time
model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs/train/exp/weights/best.pt') # works, but loads every time
model2 = model

Using cache found in C:\Users\Felix/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirements "gitpython>=3.1.30" "tqdm>=4.64.0" "setuptools>=65.5.1" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython>=3.1.30" "tqdm>=4.64.0" "setuptools>=65.5.1"  ' returned non-zero exit status 1.
YOLOv5  2023-3-13 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients
Adding AutoShape... 


## Utils

In [6]:
# Read metadata from images regarding focal length and digitial zoom
def get_focus_from_exif(f):
    with open(f, "rb") as palm_1_file:
        palm_1_image = exif(palm_1_file)
    if palm_1_image.has_exif:
        focal = palm_1_image.get('focal_length_in_35mm_film', 'Unknown')
        digital_zoom = palm_1_image.get('digital_zoom_ratio', 'Unknown')  #let's play with the zoom. the zoom starts to change the focal length and it's not needed, but let it be for now.
    else:
        focal = 0
        digital_zoom = 1
    
    return focal, digital_zoom

In [4]:
# The function spins the picture depending on what it finds in the image metadata
def exif_transpose(img):
    if not img:
        return img
    wh = img.size
    if wh[0] > wh[1]:
        return img
    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            #print('turned')
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img

In [7]:
# convert heic_to_jpg. inputs the source file and the path to the new file, returns the path to the new file
def conv_heic_to_jpg(file, new_name):

    heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
    )

    ex = heif_file.info['exif']
    image.save(new_name, format("jpeg"), exif=ex)
    image.close()
    return new_name

## Refinement methods 1

In [9]:
# procedure cuts out the license plate, just to see what is found there. I needed it for debugging.
# input a file, and a dictionary with data about the license plate
# WARNING: it will spoil the file it finds.

def crop_znak(temp_file,znak):
    if len(znak) <= 3:
        #print('не смог кроопнуть', temp_file)
        return
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    сentre_x = xmin + (xmax-xmin)/2
    сentre_y = ymin + (ymax-ymin)/2
    w_max = max(xmax-xmin, ymax-ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((сentre_x-w_max, сentre_y-w_max,сentre_x+w_max, сentre_y+w_max))
    img = img.resize((640,640))
    #print('сохраняем в ',temp_file)
    img.save(temp_file)
    #считаем коэффициент кроп
    k = znak['width']/640
    return k

In [58]:
def get_prediction(file, min_confidence):
    if '.heic' in file:
        temp_file = 'tmp.jpg'  # need a temporary file so as not to spoil the dataset
        file = conv_heic_to_jpg(file, temp_file)
    
    im = Image.open(file)
    im_width, im_height = im.size
    
    # Get prediction from model
    results = model(file)
    df = results.pandas().xyxy[0]
    df = df.loc[df['name'] == 'license_plate'] # filter for class "license_plate"
    df = df.loc[df['confidence'] >= min_confidence] # filter for confidence

    return df, im_width, im_height

In [70]:
# THIS FUNCTION SEARCHES THE DISTANCE BETWEEN TWO fixing points on a number and returns it in pixels

def res2(file, min_confidence=0.5, show_detections=False):
    df, _, _ = get_prediction(file, min_confidence)

    if show_detections: print('License plates\n', df)
    
    df = df.assign(centre_x = df.xmin + (df.xmax-df.xmin)/2)  #coordinates of the centre
    df = df.assign(centre_y = df.ymin + (df.ymax-df.ymin)/2)

    df = df.assign(width = df.xmax - df.xmin)  #width
    df = df.assign(height = df.ymax - df.ymin) #height
    
    df = df.loc[df['centre_y'] > 280] # filter 
    df = df.loc[df['centre_y'] < 380]
    
    df = df.sort_values(['centre_x']) # we sort by proximity to the centre. We need the closest number
    
    if show_detections:
        print('Sorted table\n', df)

    d = []   # there will be two points in this, we need to find the distance between them
    for _, row in df.iterrows():
        d.append([row['centre_x'], row['centre_y']])

    if len(d) == 2:
        r = ((d[0][0] - d[1][0])**2 + (d[0][1] - d[1][1])**2) **0.5
    else:
        r = 0 # i.e. it doesn't worked
         
    if show_detections: print('Distance', r)
    
    return r

In [71]:
# DEBUGGING
workfile = "../datasets/plates/test/img_1715.jpg"
res2(workfile, show_detections=True)

License plates
           xmin         ymin         xmax         ymax  confidence  class           name
0  1838.074097  1483.559326  2078.184326  1542.602661    0.757511      0  license_plate
temp1
           xmin         ymin         xmax         ymax  confidence  ...           name     centre_x     centre_y       width     height
0  1838.074097  1483.559326  2078.184326  1542.602661    0.757511  ...  license_plate  1958.129211  1513.080994  240.110229  59.043335

[1 rows x 11 columns]
filtered
 Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name, centre_x, centre_y, width, height]
Index: []

[0 rows x 11 columns]
Sorted table
 Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name, centre_x, centre_y, width, height]
Index: []

[0 rows x 11 columns]
Distance 0


0

## Refinement method 2

In [48]:
# Here we correct the sign: depending on the ratio between the found sides we find the angle and use it to calculate what 520 has become
    
from math import atan, sin, cos, tan, degrees

def correct_znak2(znak):
    
    w_znak = 529 # the width of the license plate in mm
    h_znak = 112
    if len(znak) == 3: # no license plate detected
        return w_znak
    # first look for the angle of inclination of the sign according to the aspect ratio
    HW  = 112/520 # standard ratio between the sides
    hw = znak['height']/znak['width'] # the ratio found between the sides
    tan_alfa = -(HW - hw) / (1 - HW * hw)
    alfa_rad = abs(atan(tan_alfa)) #found the angle of the license plate in radians
    print('Approximately rotated by', degrees(alfa_rad))
    
    new_AB = w_znak * cos(alfa_rad) + h_znak * sin(alfa_rad)
    #new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
    print('character length adjustment','520', '---->', new_AB)
    
    return new_AB
    

## Computing distances of test images

In [51]:
# Recognizing a file and retrieving data from it using a central number
def res(file, min_confidence=0.5, show_detections=False):
    df, width, height = get_prediction(file, min_confidence)

    if show_detections: print('License plates\n', df)

    # Keep only the license plate closest to the center
    df = df.assign(to_centre_x = abs(width/2 - (df.xmin + (df.xmax-df.xmin)/2)))  #consider the distance to the center
    df = df.assign(centre_x = df.xmin + (df.xmax-df.xmin)/2)                      #we calculate the coordinates of the center 
    
    df = df.assign(width = df.xmax - df.xmin)                                     #width
    df = df.assign(height = df.ymax - df.ymin)                                    #height
    df = df.assign(s2 = df.width * df.height)                                     #area
    
    df = df.sort_values(['to_centre_x'] )                                         #we sort by proximity to the center. We need the closest number

    # Dictionary with all results
    d = dict()
    for _, row in df.iterrows():
        d ['width']  = row['width']
        d ['height']  = row['height']
        d ['s2']  = row['s2']
        d ['xmin']  = row['xmin']
        d ['ymin']  = row['ymin']
        d ['xmax']  = row['xmax']
        d ['ymax']  = row['ymax']
        break
    # Now in d are all the characteristics of the most centered license plate
    
    d['im_width'] = width
    d['im_height'] = height
    
    return d    

In [41]:
# DEBUGGING
workfile = "../datasets/plates/test/img_1690.jpg"
res(workfile)

          xmin         ymin         xmax         ymax  confidence  ...  to_centre_x     centre_x       width     height            s2
1  1796.028442  1475.401367  2027.170776  1532.003906    0.674215  ...   104.400391  1911.599609  231.142334  56.602539  13083.242988
0   130.594742  1717.014038   428.832123  1788.838501    0.742347  ...  1736.286568   279.713432  298.237381  71.824463  21420.739703

[2 rows x 12 columns]


{'width': 231.142333984375,
 'height': 56.6025390625,
 's2': 13083.242988348007,
 'xmin': 1796.0284423828125,
 'ymin': 1475.4013671875,
 'xmax': 2027.1707763671875,
 'ymax': 1532.00390625,
 'im_width': 4032,
 'im_height': 3024}

In [53]:
def file_recognition(f):
    w_znak = 520 #the width of the license plate in mm
    #w = 4032    #sensor/photo resolution in pixels
    
    name = f[f.rfind('/')+1:]
    work_file = f
    temp_file = 'temp.jpg'
    #temp_file = 'Y:/'+name
    
    if '.heic' in f:
        temp_file = 'Y:/'+name.replace('.heic','.jpg')
        print('New name',temp_file)
        work_file = conv_heic_to_jpg(work_file,temp_file)
        #print('Converted to',work_file)

    focus, digital_zoom = get_focus_from_exif(work_file)
    print('focus, digital_zoom', focus, digital_zoom)
     
    img = Image.open(work_file)
    img = exif_transpose(img)
    w,h = img.size
    img.save(temp_file)
    
    img.close()
    
    znak1 = res(temp_file, min_confidence=0.5, show_detections=False)
    znak = znak1
    
#     # Zoom in one more time if you don't find anything. just for luck
#     if len(znak) <= 3:
#         zoom960(temp_file)
#         znak1 = res(temp_file)
#         znak = znak1
        
    
    
    #EXPERIMENT
#     if len(znak1) <= 3:
#         print('ФАЙЛ НЕ РАСПОЗНАН', f)
#         not_recognize.append(f) 
#         return 0.0
    
#     #пробуем вырезать область,чтобы знак был бллиже. количество пикселей от этого не меняется
#     print('ширина до мини пика', znak1['width'])

#     mini_pic(temp_file,znak1)
#     #вырезали и еще раз на распознавание
#     znak = res(temp_file)
#     if len(znak) != 3:
#         print('ширина после мини пика', znak['width'], 'уточнение', znak1['width']-znak['width'])
#     else: #почему то после зума, знак не распознался. Возвращаем то, что было, это всяко лучше чем ноль
#         znak = znak1
        
    #....конец эксперимента.
    
    
    w_matrix = 35 # 34.974 #is the width of the matrix in mm. This seems to be the classic width, but it is colloquially rounded up to 35 mm.
    #print('frame width',w)
    pixel_in_mm = w/w_matrix

    # REFINEMENT METHOD 1
    if 'width' in znak: # FIXME WHAT IS THIS GOOD FOR
        k = crop_znak(temp_file,znak)  #cropping ratio
        r = res2(temp_file) # got the distance between the two points on the number 
        print('r, k', r * k, znak['width'])
        if r != 0:
            r = r * k # resulted in a normal width
            #print(r)
            d1 = (1 + 487 / (r/pixel_in_mm)) * focus/1000
            print('point-to-point distance', d1)
            return d1 # if there is distance between points, it is advantageous
    
    print(znak)
    if len(znak) <= 3:
        print('A! NO LICENSE PLATE DETECTED', f)
        #not_recognized.append(f)
        d = 0.0
    
    # REFINEMENT METHOD 2
    else:
        w_znak = correct_znak2(znak)
        d = (1 + w_znak /(znak['width']/pixel_in_mm))* focus/1000  #I couldn't decide whether to count to the optical centre or to the sensor. I decided to count to the sensor. The lens can move away from the sensor by many centimetres.
    
    return d

In [23]:
# DEBUGGING
workfile = "../datasets/plates/test/img_1628.jpg"

file_recognition(workfile)

focus, digital_zoom 0 1
{'im_width': 4032, 'im_height': 3024}
A! FILE NOT RECOGNISED ../datasets/plates/test/img_1628.jpg


0.0

In [52]:
train_csv = '../data_set/train.csv'  #data set if you run it on a training dataset, it will be considered an error.
keys = ['width', 'height', 's2', 'xmin', 'ymin', 'xmax', 'ymax','im_width', 'im_height']
pic_data = '../datasets/plates/train'  # here is the dataset
pic_data_test  = pic_data

# TO RUN ON YOUR DATASET, YOU NEED TO CHANGE THIS PATH
pic_data_test = '../datasets/plates/test'  # Here is the test dataset

test = '../data_set/custom_solution_v1.csv'  # the result will be recorded here

# first read all the distances given to us from the training dataset
dist = dict()  # here we will store all distances extracted from the file. In the form of a dictionary
with open(train_csv, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.replace('\n','')
        key, d = line.split(';')
        dist[key] = d
        
    

new_f = 'image_name;distance'
not_recognized = []  # there will be some unrecognized
n = 0
abs_mistake, otn_mistake, vsego = 0, 0, 0  # To calculate the error

test_images = os.listdir(pic_data_test)
test_images = [img for img in test_images if img.endswith(".jpg")]


# Iterate through all test images
for f in test_images:
    n += 1
#    if n<= 380:
#        continue
#    if n>385:
#         break
    
    
    #work_file = os.path.join(pic_data_test, f) # FIXME uncomment!
    work_file = pic_data_test + "/" + f
    print(n, 'working with the file: ', work_file)

    # Here the magic happens, get prediction
    itog = file_recognition(work_file)
    if itog == 0:
        not_recognized.append(work_file)
    
    # we get the true value, count the error
    if f in dist:
        y = float(dist[f])
        mistake = (y - itog) / y # I apologize for not using your mistake, but it makes more sense to me in percentages, and since I don't train on a mistake, I did so.
        print('Error', mistake, 'true distance', y, 'prediction', itog)
        otn_mistake += mistake  #you need it to see if it goes in the plus or minus
        abs_mistake += abs(mistake)
        vsego += 1
    
    
    st = f + ';' + str(itog)
    print('Recorded result: ', st)
    print('\n\n')    
    new_f = new_f +'\n'+st
    
# Save predictions in csv file
with open(test, 'w', encoding = 'utf-8') as file:
    file.write(new_f)
print('Nr of files not detected:', len(not_recognized))
print('unrecognized:', *not_recognized)
print('saved in:', test)
if vsego != 0:
    print('Аbsolute error', round(abs_mistake/vsego*100, 5))
    print('Relative error', round(otn_mistake/vsego*100, 5))
else:
    print('No error has been calculated. No true data detected. Is the dataset a test one?')

1 working with the file:  ../datasets/plates/test/img_1628.jpg
focus, digital_zoom 18 1.3795620437956204
{'im_width': 4032, 'im_height': 3024}
A! NO LICENSE PLATE DETECTED ../datasets/plates/test/img_1628.jpg
Error 1.0 true distance 1.12 prediction 0.0
Recorded result:  img_1628.jpg;0.0



2 working with the file:  ../datasets/plates/test/img_1690.jpg
focus, digital_zoom 16 1.237315875613748
сохраняем в  temp.jpg
r,k 0.0 231.1754150390625
{'width': 231.1754150390625, 'height': 56.4276123046875, 's2': 13044.676694199443, 'xmin': 1795.8594970703125, 'ymin': 1475.58349609375, 'xmax': 2027.034912109375, 'ymax': 1532.0111083984375, 'im_width': 4032, 'im_height': 3024}
Approximately rotated by 1.735433024586337
character length adjustment 520 ----> 532.1492112876026
Error 0.009554862273147552 true distance 4.3 prediction 4.258914092225465
Recorded result:  img_1690.jpg;4.258914092225465



3 working with the file:  ../datasets/plates/test/img_1715.jpg
focus, digital_zoom 16 1.237315875613748

## Experiments

In [10]:
# EXPERIMENTAL UTILS
# Cut out the area around the sign to feed it back into the network for more accurate recognition. 
def mini_pic(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    if znak['width'] > 400: # only if the sign is very small
        return
    
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    centre_x = xmin + (xmax - xmin)/2
    centre_y = ymin + (ymax - ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
    #print('минипик',temp_file,centre_x,centre_y, znak['width'])
    
# try to cut out the area around the sign to feed it back to the network for more accurate recognition.  
# This is if the sign in the photo is small and it did not find anything at all. do a cropped center 960x960
def zoom960(temp_file):
    
    img = Image.open(temp_file)
    w,h = img.size
    centre_x = w//2
    centre_y = h//2
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
 

In [56]:
res('Y:\\255_1.jpg',1)

Знаки
           xmin         ymin         xmax         ymax  confidence  class  name
0  1760.632202  1313.192139  2381.237305  1438.997437    0.952805      0  znak


{'width': 620.6051025390625,
 'height': 125.8052978515625,
 's2': 78075.40977312624,
 'xmin': 1760.6322021484375,
 'ymin': 1313.192138671875,
 'xmax': 2381.2373046875,
 'ymax': 1438.9974365234375,
 'im_width': 3968,
 'im_height': 2976}

In [166]:
file_recognition('..\\data_set\\test\\img_2674.heic')

Новое имя Y:\img_2674.jpg
focus, digital_zoom 14 1.0327868852459017
ФАЙЛ НЕ РАСПОЗНАН ..\data_set\test\img_2674.heic


0.0

In [161]:
print(file_recognition('Y:\\255_1.jpg'))

print('\n')
print(file_recognition('Y:\\255_2.jpg'))


print('\n')

print(file_recognition('Y:\\255_3.jpg'))


print('\n')

focus, digital_zoom 27 1.0
сохраняем в  temp.jpg
r,k 592.7320572254057 619.9266357421875
расстояние по точкам 2.5419996463221812
2.5419996463221812


focus, digital_zoom 31 1.0
сохраняем в  temp.jpg
r,k 718.0005061784991 743.150390625
расстояние по точкам 2.414798399046464
2.414798399046464


focus, digital_zoom 52 1.0
сохраняем в  temp.jpg
r,k 1166.430252539312 1251.6114501953125
расстояние по точкам 2.513371394382705
2.513371394382705




In [160]:
print(file_recognition('Y:\\421_1.jpg'))
print('\n')
print(file_recognition('Y:\\421_2.jpg'))
print('\n')

print(file_recognition('Y:\\421_3.jpg'))
print('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'Y:\\421_1.jpg'

In [91]:
print(file_recognition('Y:\\490_1.jpg'))
print('\n')
print(file_recognition('Y:\\490_2.jpg'))
print('\n')

print(file_recognition('Y:\\490_3.jpg'))
print('\n')

4.923146672370446


4.690990338609638


4.974977451052313


